In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, utils, Input, Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Define constants for image size, batch size, and epochs
IMAGE_SIZE      = (180, 180)
BATCH_SIZE      = 32
EPOCHS          = 10
TRANSFER_EPOCHS = 10
DATASET_PATH    = 'Dataset'

In [ ]:
# Download and unzip the cats and dogs dataset
!wget -q https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip
# Clean up unnecessary files and rename the directory
!rm kagglecatsanddogs_5340.zip readme\[1\].txt CDLA-Permissive-2.0.pdf
!mv PetImages Dataset

In [ ]:
# Create the training dataset from the directory, splitting 20% for validation
train_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
# Create the validation dataset from the directory
val_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
# Get the number of classes from the training dataset
num_classes = len(train_ds.class_names)
num_classes

In [ ]:
# Define the custom CNN model architecture
model = models.Sequential([
    Input(shape=IMAGE_SIZE + (3,)),
    layers.Rescaling(1./255), # Normalize pixel values to [0, 1]
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2), # Add dropout for regularization
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax') # Output layer
])

In [ ]:
# Compile the model with Adam optimizer and sparse categorical crossentropy loss
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Train the model and store the history
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

In [ ]:
# Plot the training and validation accuracy for the custom CNN
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"],     label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.legend()
plt.title("Custom CNN Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

In [ ]:
# Create a convolutional base from the first few layers of the trained model
conv_base = models.Sequential()
for layer in model.layers[:4]:  # Input, Rescaling, Conv2D, MaxPooling2D
    conv_base.add(layer)

# Freeze the convolutional base to prevent its weights from being updated
conv_base.trainable = False

In [ ]:
# Build a new model for transfer learning
inputs = Input(shape=IMAGE_SIZE + (3,))
x = conv_base(inputs, training=False) # Pass inputs through the frozen base
x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x) # Add new trainable dense layers
outputs = layers.Dense(num_classes, activation='softmax')(x)

transfer_model = Model(inputs, outputs)

In [ ]:
# Print the summary of the original model (to compare)
model.summary()

In [ ]:
# Compile the transfer learning model
transfer_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the transfer learning model
transfer_history = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=TRANSFER_EPOCHS
)

In [ ]:
# Plot the training and validation accuracy for the transfer learning model
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 2)
plt.plot(transfer_history.history["accuracy"], label="train_acc")
plt.plot(transfer_history.history["val_accuracy"], label="val_acc")
plt.legend()
plt.title("Transfer Learning Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()